In [2]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool  # Import global_mean_pool
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.loader import DataLoader

dataset = MNISTSuperpixels(root='./data/MNISTSuperpixels')

train_dataset = dataset[:50000]
test_dataset = dataset[50000:]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels=1, out_channels=32)
        self.conv2 = GCNConv(in_channels=32, out_channels=64)
        self.fc1 = torch.nn.Linear(64, 32)
        self.fc2 = torch.nn.Linear(32, 10)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train(model, loader):
    model.train()
    total_loss = 0
    correct = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)

def test(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

for epoch in range(1, 11):
    train_loss, train_acc = train(model, train_loader)
    test_acc = test(model, test_loader)
    print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')


C:\Users\deepa\anaconda3\Lib\site-packages\torch_geometric\data\dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f) != _repr(sel

Epoch 1, Train Loss: 2.1499, Train Acc: 0.1938, Test Acc: 0.2428
Epoch 2, Train Loss: 1.9978, Train Acc: 0.2510, Test Acc: 0.2366
Epoch 3, Train Loss: 1.9823, Train Acc: 0.2604, Test Acc: 0.2733
Epoch 4, Train Loss: 1.9737, Train Acc: 0.2655, Test Acc: 0.2772
Epoch 5, Train Loss: 1.9559, Train Acc: 0.2708, Test Acc: 0.2872
Epoch 6, Train Loss: 1.9489, Train Acc: 0.2750, Test Acc: 0.2871
Epoch 7, Train Loss: 1.9388, Train Acc: 0.2779, Test Acc: 0.2994
Epoch 8, Train Loss: 1.9409, Train Acc: 0.2775, Test Acc: 0.2827
Epoch 9, Train Loss: 1.9281, Train Acc: 0.2828, Test Acc: 0.2996
Epoch 10, Train Loss: 1.9297, Train Acc: 0.2811, Test Acc: 0.2941
